# High-Throughput Meta-Atom Defect Classification via K-Means Clustering

**Automated defect detection in metasurface arrays using unsupervised machine learning**

This notebook demonstrates a scalable pipeline for:
1. Automated segmentation of metasurface arrays into individual meta-atoms
2. Multi-dimensional feature extraction
3. K-Means clustering for defect categorization
4. Statistical validation and visualization

In [ ]:
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
from matplotlib.patches import Rectangle, Circle
from matplotlib.colors import LinearSegmentedColormap
import glob
from pathlib import Path
from collections import Counter

from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
from sklearn.manifold import TSNE
from sklearn.metrics import silhouette_score, silhouette_samples

import warnings
warnings.filterwarnings('ignore')

np.random.seed(42)

# Publication-quality plot settings
plt.rcParams.update({
    'font.size': 11,
    'font.family': 'sans-serif',
    'axes.labelsize': 12,
    'axes.titlesize': 13,
    'xtick.labelsize': 10,
    'ytick.labelsize': 10,
    'legend.fontsize': 10,
    'figure.dpi': 150,
    'savefig.dpi': 300,
    'savefig.bbox': 'tight',
    'axes.spines.top': False,
    'axes.spines.right': False
})

# Color palette for clusters
CLUSTER_COLORS = ['#2ecc71', '#e74c3c', '#3498db', '#9b59b6', '#f39c12']
CLUSTER_NAMES = ['Intact', 'Missing', 'Collapsed', 'Irregular']

BASE_DIR = Path(os.getcwd())
if BASE_DIR.name == 'notebooks':
    BASE_DIR = BASE_DIR.parent
OUTPUT_DIR = BASE_DIR / "Meta_Atoms"
FIGURES_DIR = BASE_DIR / "figures"
OUTPUT_DIR.mkdir(exist_ok=True)
FIGURES_DIR.mkdir(exist_ok=True)

print(f"Working directory: {BASE_DIR}")

---
## 1. Array Segmentation Pipeline

Automated extraction of 441 meta-atom tiles from each 21x21 metasurface array.

In [ ]:
def segment_array(image_path, output_dir, grid_size=21, tile_size=32, show_grid=False):
    """Extract individual meta-atom tiles from array image."""
    img = cv2.imread(str(image_path))
    if img is None:
        raise ValueError(f"Could not read {image_path}")
    
    H, W = img.shape[:2]
    base_name = os.path.splitext(os.path.basename(image_path))[0]
    array_dir = os.path.join(output_dir, base_name)
    os.makedirs(array_dir, exist_ok=True)
    
    x_spacing = W / grid_size
    y_spacing = H / grid_size
    
    tiles = []
    for r in range(grid_size):
        for c in range(grid_size):
            cx = int((c + 0.5) * x_spacing)
            cy = int((r + 0.5) * y_spacing)
            x1, y1 = max(0, cx - tile_size), max(0, cy - tile_size)
            x2, y2 = min(W, cx + tile_size), min(H, cy + tile_size)
            
            tile = img[y1:y2, x1:x2]
            fname = f"{base_name}_{r+1},{c+1}.bmp"
            cv2.imwrite(os.path.join(array_dir, fname), tile)
            tiles.append({'row': r+1, 'col': c+1, 'center': (cx, cy)})
    
    return img, tiles, (x_spacing, y_spacing)

# Process all arrays
arrays = ["Array_1Crop.bmp", "Array_2Crop.bmp", "Array_3Crop.bmp"]
array_images = {}

for arr in arrays:
    arr_path = BASE_DIR / arr
    if arr_path.exists():
        img, tiles, spacing = segment_array(str(arr_path), str(OUTPUT_DIR), show_grid=False)
        array_images[arr.replace('.bmp', '')] = img
        print(f"{arr}: Extracted {len(tiles)} tiles")

### Figure 1: Segmentation Overview

In [ ]:
# Create segmentation overview figure
fig = plt.figure(figsize=(14, 5))
gs = gridspec.GridSpec(1, 4, width_ratios=[1, 1, 1, 0.4], wspace=0.15)

for idx, (name, img) in enumerate(array_images.items()):
    ax = fig.add_subplot(gs[idx])
    img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    ax.imshow(img_rgb)
    
    # Draw grid overlay
    H, W = img.shape[:2]
    for i in range(22):
        ax.axhline(y=i * H/21, color='lime', linewidth=0.3, alpha=0.5)
        ax.axvline(x=i * W/21, color='lime', linewidth=0.3, alpha=0.5)
    
    ax.set_title(f'{name.replace("Crop", "")}\n(21 x 21 = 441 atoms)', fontsize=11)
    ax.axis('off')

# Info panel
ax_info = fig.add_subplot(gs[3])
ax_info.axis('off')
info_text = (
    "Segmentation\nStatistics\n"
    "----------\n"
    f"Arrays: 3\n"
    f"Grid: 21 x 21\n"
    f"Atoms/array: 441\n"
    f"Total atoms: 1323\n"
    f"Tile size: 64x64 px"
)
ax_info.text(0.1, 0.5, info_text, transform=ax_info.transAxes, 
             fontsize=11, verticalalignment='center', fontfamily='monospace',
             bbox=dict(boxstyle='round', facecolor='#f0f0f0', edgecolor='gray'))

plt.suptitle('Automated Metasurface Array Segmentation Pipeline', fontsize=14, fontweight='bold', y=1.02)
plt.savefig(FIGURES_DIR / 'fig1_segmentation_overview.png', dpi=300, bbox_inches='tight')
plt.show()

---
## 2. Feature Extraction

In [ ]:
def extract_features(img_gray):
    """Extract comprehensive feature vector from meta-atom image."""
    # Gradient features
    sobelx = cv2.Sobel(img_gray, cv2.CV_64F, 1, 0, ksize=3)
    sobely = cv2.Sobel(img_gray, cv2.CV_64F, 0, 1, ksize=3)
    mag = np.sqrt(sobelx**2 + sobely**2)
    
    # Intensity features
    mean_intensity = np.mean(img_gray)
    std_intensity = np.std(img_gray)
    
    # Binary segmentation
    img_bin = cv2.adaptiveThreshold(img_gray, 255, cv2.ADAPTIVE_THRESH_MEAN_C,
                                     cv2.THRESH_BINARY_INV, 21, 3)
    
    # Contour features
    contours, _ = cv2.findContours(img_bin, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    
    area, perimeter, solidity, circularity = 0, 0, 0, 0
    hu_moments = np.zeros(7)
    
    if contours:
        cnt = max(contours, key=cv2.contourArea)
        area = cv2.contourArea(cnt)
        perimeter = cv2.arcLength(cnt, True)
        
        hull = cv2.convexHull(cnt)
        hull_area = cv2.contourArea(hull)
        solidity = area / hull_area if hull_area > 0 else 0
        circularity = (4 * np.pi * area) / (perimeter ** 2) if perimeter > 0 else 0
        
        mask = np.zeros_like(img_bin)
        cv2.drawContours(mask, [cnt], -1, 255, -1)
        hu_moments = cv2.HuMoments(cv2.moments(mask)).flatten()
    
    return {
        'mean_intensity': mean_intensity,
        'std_intensity': std_intensity,
        'max_gradient': np.max(mag),
        'mean_gradient': np.mean(mag),
        'area': area,
        'solidity': solidity,
        'circularity': circularity,
        'hu_moments': hu_moments,
        'laplacian_var': cv2.Laplacian(img_gray, cv2.CV_64F).var()
    }

def load_all_tiles():
    """Load all tiles and extract features."""
    all_data = []
    
    for array_name in ["Array_1Crop", "Array_2Crop", "Array_3Crop"]:
        array_dir = OUTPUT_DIR / array_name
        if not array_dir.exists():
            continue
        
        for fpath in array_dir.glob("*.bmp"):
            img = cv2.imread(str(fpath), cv2.IMREAD_GRAYSCALE)
            if img is None:
                continue
            
            fname = fpath.stem
            try:
                coords = fname.split('_')[-1]
                row, col = map(int, coords.split(','))
            except:
                row, col = -1, -1
            
            features = extract_features(img)
            all_data.append({
                'array': array_name,
                'filename': fpath.name,
                'filepath': str(fpath),
                'row': row, 'col': col,
                'image': img,
                **features
            })
    
    return all_data

# Load data
tile_data = load_all_tiles()
print(f"Total meta-atoms loaded: {len(tile_data)}")

In [ ]:
def build_feature_matrix(tile_data):
    """Construct feature matrix for clustering."""
    features = []
    for t in tile_data:
        row = [
            t['mean_intensity'], t['std_intensity'],
            t['max_gradient'], t['mean_gradient'],
            t['area'], t['solidity'], t['circularity'],
            t['laplacian_var']
        ]
        hu_log = -np.sign(t['hu_moments']) * np.log10(np.abs(t['hu_moments']) + 1e-10)
        row.extend(hu_log)
        features.append(row)
    
    X = np.nan_to_num(np.array(features), nan=0, posinf=0, neginf=0)
    return X

X_raw = build_feature_matrix(tile_data)
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X_raw)

# PCA
pca = PCA(n_components=5)
X_pca = pca.fit_transform(X_scaled)

print(f"Feature matrix: {X_raw.shape}")
print(f"PCA explained variance: {pca.explained_variance_ratio_.sum():.1%}")

### Figure 2: Feature Space Visualization

In [ ]:
# t-SNE embedding
print("Computing t-SNE embedding...")
tsne = TSNE(n_components=2, perplexity=30, random_state=42, n_iter=1000)
X_tsne = tsne.fit_transform(X_pca)

fig, axes = plt.subplots(1, 3, figsize=(14, 4))

# PCA scatter
ax = axes[0]
scatter = ax.scatter(X_pca[:, 0], X_pca[:, 1], c=X_raw[:, 0], cmap='viridis', 
                      alpha=0.6, s=15, edgecolors='none')
ax.set_xlabel('PC1')
ax.set_ylabel('PC2')
ax.set_title('PCA Projection\n(colored by intensity)')
plt.colorbar(scatter, ax=ax, label='Mean Intensity')

# t-SNE scatter
ax = axes[1]
scatter = ax.scatter(X_tsne[:, 0], X_tsne[:, 1], c=X_raw[:, 4], cmap='plasma',
                      alpha=0.6, s=15, edgecolors='none')
ax.set_xlabel('t-SNE 1')
ax.set_ylabel('t-SNE 2')
ax.set_title('t-SNE Embedding\n(colored by area)')
plt.colorbar(scatter, ax=ax, label='Area')

# Feature correlations
ax = axes[2]
feature_names = ['Intensity', 'Std', 'MaxGrad', 'MeanGrad', 'Area', 'Solidity', 'Circular', 'Laplacian']
corr_matrix = np.corrcoef(X_raw[:, :8].T)
im = ax.imshow(corr_matrix, cmap='RdBu_r', vmin=-1, vmax=1)
ax.set_xticks(range(8))
ax.set_yticks(range(8))
ax.set_xticklabels(feature_names, rotation=45, ha='right', fontsize=8)
ax.set_yticklabels(feature_names, fontsize=8)
ax.set_title('Feature Correlation Matrix')
plt.colorbar(im, ax=ax, fraction=0.046)

plt.suptitle('Feature Space Analysis', fontsize=14, fontweight='bold', y=1.02)
plt.tight_layout()
plt.savefig(FIGURES_DIR / 'fig2_feature_space.png', dpi=300, bbox_inches='tight')
plt.show()

---
## 3. K-Means Clustering Analysis

### Figure 3: Optimal K Selection

In [ ]:
k_range = range(2, 10)
inertias = []
silhouettes = []

for k in k_range:
    km = KMeans(n_clusters=k, random_state=42, n_init=10)
    labels = km.fit_predict(X_pca)
    inertias.append(km.inertia_)
    silhouettes.append(silhouette_score(X_pca, labels))

fig, axes = plt.subplots(1, 3, figsize=(14, 4))

# Elbow plot
ax = axes[0]
ax.plot(k_range, inertias, 'o-', color='#2c3e50', linewidth=2, markersize=8)
ax.axvline(x=4, color='#e74c3c', linestyle='--', linewidth=2, label='K=4')
ax.set_xlabel('Number of Clusters (K)')
ax.set_ylabel('Inertia (Within-Cluster SS)')
ax.set_title('Elbow Method')
ax.legend()
ax.set_xticks(range(2, 10))

# Silhouette plot
ax = axes[1]
ax.plot(k_range, silhouettes, 's-', color='#27ae60', linewidth=2, markersize=8)
ax.axvline(x=4, color='#e74c3c', linestyle='--', linewidth=2, label='K=4')
ax.set_xlabel('Number of Clusters (K)')
ax.set_ylabel('Silhouette Score')
ax.set_title('Silhouette Analysis')
ax.legend()
ax.set_xticks(range(2, 10))

# Combined score
ax = axes[2]
# Normalize both metrics
inertias_norm = (np.array(inertias) - min(inertias)) / (max(inertias) - min(inertias))
silhouettes_norm = np.array(silhouettes)
combined = silhouettes_norm - 0.3 * inertias_norm  # Weight silhouette more

ax.bar(k_range, combined, color=[CLUSTER_COLORS[i % len(CLUSTER_COLORS)] for i in range(len(k_range))],
       edgecolor='black', linewidth=0.5)
ax.axhline(y=combined[2], color='#e74c3c', linestyle='--', linewidth=2)  # K=4 line
ax.set_xlabel('Number of Clusters (K)')
ax.set_ylabel('Combined Score')
ax.set_title('Optimal K Selection')
ax.set_xticks(range(2, 10))

best_k = list(k_range)[np.argmax(silhouettes)]
print(f"Optimal K by silhouette: {best_k}")

plt.suptitle('K-Means: Optimal Cluster Selection', fontsize=14, fontweight='bold', y=1.02)
plt.tight_layout()
plt.savefig(FIGURES_DIR / 'fig3_optimal_k.png', dpi=300, bbox_inches='tight')
plt.show()

In [ ]:
# Final K-Means with K=4
K = 4
kmeans = KMeans(n_clusters=K, random_state=42, n_init=20)
labels = kmeans.fit_predict(X_pca)

# Store cluster labels
for i, tile in enumerate(tile_data):
    tile['cluster'] = labels[i]

# Cluster statistics
cluster_counts = Counter(labels)
print("\nCluster Distribution:")
for c in range(K):
    print(f"  Cluster {c}: {cluster_counts[c]} atoms ({cluster_counts[c]/len(labels)*100:.1f}%)")

print(f"\nSilhouette Score: {silhouette_score(X_pca, labels):.3f}")

### Figure 4: Clustering Results in Feature Space

In [ ]:
fig, axes = plt.subplots(1, 3, figsize=(15, 5))

# t-SNE with clusters
ax = axes[0]
for c in range(K):
    mask = labels == c
    ax.scatter(X_tsne[mask, 0], X_tsne[mask, 1], c=CLUSTER_COLORS[c], 
               label=f'Cluster {c} (n={mask.sum()})', alpha=0.7, s=20, edgecolors='white', linewidth=0.3)
ax.set_xlabel('t-SNE 1')
ax.set_ylabel('t-SNE 2')
ax.set_title('t-SNE Clustering')
ax.legend(loc='best', framealpha=0.9)

# PCA with clusters and centroids
ax = axes[1]
for c in range(K):
    mask = labels == c
    ax.scatter(X_pca[mask, 0], X_pca[mask, 1], c=CLUSTER_COLORS[c], 
               label=f'Cluster {c}', alpha=0.5, s=15)
# Plot centroids
centroids_pca = kmeans.cluster_centers_
ax.scatter(centroids_pca[:, 0], centroids_pca[:, 1], c='black', marker='X', 
           s=200, edgecolors='white', linewidth=2, label='Centroids', zorder=10)
ax.set_xlabel('PC1')
ax.set_ylabel('PC2')
ax.set_title('PCA Clustering with Centroids')
ax.legend(loc='best', framealpha=0.9)

# Silhouette analysis
ax = axes[2]
silhouette_vals = silhouette_samples(X_pca, labels)
y_lower = 10

for c in range(K):
    cluster_silhouette = silhouette_vals[labels == c]
    cluster_silhouette.sort()
    size_cluster = cluster_silhouette.shape[0]
    y_upper = y_lower + size_cluster
    
    ax.fill_betweenx(np.arange(y_lower, y_upper), 0, cluster_silhouette,
                      facecolor=CLUSTER_COLORS[c], edgecolor=CLUSTER_COLORS[c], alpha=0.7)
    ax.text(-0.05, y_lower + 0.5 * size_cluster, str(c), fontsize=10, fontweight='bold')
    y_lower = y_upper + 10

ax.axvline(x=silhouette_score(X_pca, labels), color='red', linestyle='--', linewidth=2, label='Avg Score')
ax.set_xlabel('Silhouette Coefficient')
ax.set_ylabel('Cluster')
ax.set_title('Silhouette Plot')
ax.set_xlim([-0.1, 1])
ax.legend()

plt.suptitle('K-Means Clustering Results (K=4)', fontsize=14, fontweight='bold', y=1.02)
plt.tight_layout()
plt.savefig(FIGURES_DIR / 'fig4_clustering_results.png', dpi=300, bbox_inches='tight')
plt.show()

---
## 4. Cluster Sample Gallery

Representative examples from each cluster.

### Figure 5: Cluster Sample Grid (Large)

In [ ]:
def show_cluster_gallery(tile_data, n_samples=20):
    """Display large gallery of samples from each cluster."""
    n_clusters = K
    n_cols = n_samples
    
    fig, axes = plt.subplots(n_clusters, n_cols, figsize=(n_cols * 0.8, n_clusters * 1.2))
    
    for c in range(n_clusters):
        cluster_tiles = [t for t in tile_data if t['cluster'] == c]
        np.random.shuffle(cluster_tiles)
        samples = cluster_tiles[:n_samples]
        
        for j in range(n_cols):
            ax = axes[c, j]
            if j < len(samples):
                ax.imshow(samples[j]['image'], cmap='gray')
            ax.axis('off')
            
            # Row label
            if j == 0:
                ax.set_ylabel(f'C{c}\n(n={len(cluster_tiles)})', fontsize=10, fontweight='bold',
                             rotation=0, labelpad=30, va='center', color=CLUSTER_COLORS[c])
    
    plt.suptitle('Cluster Sample Gallery: 20 Random Samples per Cluster', fontsize=14, fontweight='bold', y=1.01)
    plt.tight_layout()
    plt.savefig(FIGURES_DIR / 'fig5_cluster_gallery.png', dpi=300, bbox_inches='tight')
    plt.show()

show_cluster_gallery(tile_data, n_samples=20)

### Figure 6: Extended Cluster Examples (50 per cluster)

In [ ]:
def show_extended_gallery(tile_data, cluster_id, n_samples=50):
    """Show many examples from a single cluster."""
    cluster_tiles = [t for t in tile_data if t['cluster'] == cluster_id]
    np.random.shuffle(cluster_tiles)
    samples = cluster_tiles[:n_samples]
    
    n_cols = 10
    n_rows = (n_samples + n_cols - 1) // n_cols
    
    fig, axes = plt.subplots(n_rows, n_cols, figsize=(n_cols * 1.2, n_rows * 1.2))
    axes = axes.flatten()
    
    for i, ax in enumerate(axes):
        if i < len(samples):
            ax.imshow(samples[i]['image'], cmap='gray')
            ax.set_title(f"{samples[i]['row']},{samples[i]['col']}", fontsize=6, pad=1)
        ax.axis('off')
    
    plt.suptitle(f'Cluster {cluster_id}: {len(cluster_tiles)} Total Atoms ({n_samples} shown)',
                 fontsize=14, fontweight='bold', color=CLUSTER_COLORS[cluster_id], y=1.01)
    plt.tight_layout()
    return fig

# Show each cluster
for c in range(K):
    fig = show_extended_gallery(tile_data, c, n_samples=50)
    fig.savefig(FIGURES_DIR / f'fig6_cluster{c}_samples.png', dpi=300, bbox_inches='tight')
    plt.show()

---
## 5. Spatial Distribution Analysis

### Figure 7: Cluster Heatmaps on Array

In [ ]:
def create_cluster_map(tile_data, array_name):
    """Create spatial heatmap of clusters on array grid."""
    grid = np.full((21, 21), -1)
    
    for tile in tile_data:
        if tile['array'] == array_name and tile['row'] > 0 and tile['col'] > 0:
            grid[tile['row']-1, tile['col']-1] = tile['cluster']
    
    return grid

fig, axes = plt.subplots(1, 3, figsize=(14, 5))

# Custom colormap for clusters
cmap = LinearSegmentedColormap.from_list('clusters', CLUSTER_COLORS[:K], N=K)

for idx, array_name in enumerate(["Array_1Crop", "Array_2Crop", "Array_3Crop"]):
    ax = axes[idx]
    grid = create_cluster_map(tile_data, array_name)
    
    im = ax.imshow(grid, cmap=cmap, vmin=0, vmax=K-1)
    ax.set_title(f'{array_name.replace("Crop", "")}', fontsize=12)
    ax.set_xlabel('Column')
    ax.set_ylabel('Row')
    
    # Grid lines
    ax.set_xticks(np.arange(-0.5, 21, 1), minor=True)
    ax.set_yticks(np.arange(-0.5, 21, 1), minor=True)
    ax.grid(which='minor', color='white', linewidth=0.5, alpha=0.5)
    ax.tick_params(which='minor', size=0)

# Colorbar
cbar = fig.colorbar(im, ax=axes, shrink=0.6, pad=0.02)
cbar.set_ticks(np.arange(K))
cbar.set_ticklabels([f'C{i}' for i in range(K)])

plt.suptitle('Spatial Distribution of Defect Clusters', fontsize=14, fontweight='bold', y=1.02)
plt.savefig(FIGURES_DIR / 'fig7_spatial_distribution.png', dpi=300, bbox_inches='tight')
plt.show()

### Figure 8: Cluster Feature Distributions

In [ ]:
fig, axes = plt.subplots(2, 3, figsize=(14, 8))
axes = axes.flatten()

features_to_plot = [
    ('mean_intensity', 'Mean Intensity'),
    ('area', 'Contour Area'),
    ('solidity', 'Solidity'),
    ('circularity', 'Circularity'),
    ('max_gradient', 'Max Gradient'),
    ('std_intensity', 'Intensity Std Dev')
]

for idx, (feat_key, feat_name) in enumerate(features_to_plot):
    ax = axes[idx]
    
    data_by_cluster = []
    for c in range(K):
        cluster_vals = [t[feat_key] for t in tile_data if t['cluster'] == c]
        data_by_cluster.append(cluster_vals)
    
    bp = ax.boxplot(data_by_cluster, patch_artist=True, widths=0.6)
    
    for patch, color in zip(bp['boxes'], CLUSTER_COLORS[:K]):
        patch.set_facecolor(color)
        patch.set_alpha(0.7)
    
    ax.set_xticklabels([f'C{c}' for c in range(K)])
    ax.set_xlabel('Cluster')
    ax.set_ylabel(feat_name)
    ax.set_title(feat_name)

plt.suptitle('Feature Distributions by Cluster', fontsize=14, fontweight='bold', y=1.02)
plt.tight_layout()
plt.savefig(FIGURES_DIR / 'fig8_feature_distributions.png', dpi=300, bbox_inches='tight')
plt.show()

---
## 6. Summary Statistics

### Figure 9: Pipeline Summary

In [ ]:
fig = plt.figure(figsize=(16, 10))
gs = gridspec.GridSpec(3, 4, height_ratios=[1.2, 1, 1], hspace=0.3, wspace=0.25)

# Row 1: Pipeline overview
ax_pipe = fig.add_subplot(gs[0, :])
ax_pipe.axis('off')

# Draw pipeline boxes
steps = [
    ('Input\nArrays', '#3498db', '3 BMP\nImages'),
    ('Segmentation', '#2ecc71', '1323\nTiles'),
    ('Feature\nExtraction', '#f39c12', '15 Features\nper tile'),
    ('K-Means\nClustering', '#9b59b6', '4 Clusters'),
    ('Classification', '#e74c3c', 'Defect\nLabels')
]

for i, (label, color, detail) in enumerate(steps):
    x = 0.1 + i * 0.18
    rect = plt.Rectangle((x, 0.3), 0.14, 0.4, transform=ax_pipe.transAxes,
                          facecolor=color, edgecolor='black', linewidth=2, alpha=0.8)
    ax_pipe.add_patch(rect)
    ax_pipe.text(x + 0.07, 0.5, label, transform=ax_pipe.transAxes,
                 ha='center', va='center', fontsize=11, fontweight='bold', color='white')
    ax_pipe.text(x + 0.07, 0.2, detail, transform=ax_pipe.transAxes,
                 ha='center', va='center', fontsize=9, style='italic')
    
    if i < len(steps) - 1:
        ax_pipe.annotate('', xy=(x + 0.16, 0.5), xytext=(x + 0.14, 0.5),
                         xycoords='axes fraction', textcoords='axes fraction',
                         arrowprops=dict(arrowstyle='->', color='black', lw=2))

ax_pipe.set_title('High-Throughput Meta-Atom Defect Classification Pipeline', fontsize=14, fontweight='bold')

# Row 2: Cluster pie chart and bar chart
ax_pie = fig.add_subplot(gs[1, 0])
sizes = [cluster_counts[c] for c in range(K)]
explode = [0.02] * K
ax_pie.pie(sizes, explode=explode, labels=[f'C{c}' for c in range(K)], colors=CLUSTER_COLORS[:K],
           autopct='%1.1f%%', shadow=False, startangle=90)
ax_pie.set_title('Cluster Distribution')

ax_bar = fig.add_subplot(gs[1, 1])
ax_bar.bar(range(K), sizes, color=CLUSTER_COLORS[:K], edgecolor='black', linewidth=1)
ax_bar.set_xticks(range(K))
ax_bar.set_xticklabels([f'C{c}' for c in range(K)])
ax_bar.set_ylabel('Count')
ax_bar.set_title('Cluster Counts')
for i, v in enumerate(sizes):
    ax_bar.text(i, v + 10, str(v), ha='center', fontsize=10, fontweight='bold')

# Per-array breakdown
ax_stacked = fig.add_subplot(gs[1, 2:])
array_names = ["Array_1Crop", "Array_2Crop", "Array_3Crop"]
bottom = np.zeros(3)

for c in range(K):
    counts = [sum(1 for t in tile_data if t['array'] == arr and t['cluster'] == c) for arr in array_names]
    ax_stacked.bar(range(3), counts, bottom=bottom, label=f'C{c}', color=CLUSTER_COLORS[c])
    bottom += counts

ax_stacked.set_xticks(range(3))
ax_stacked.set_xticklabels(['Array 1', 'Array 2', 'Array 3'])
ax_stacked.set_ylabel('Count')
ax_stacked.set_title('Cluster Distribution by Array')
ax_stacked.legend(loc='upper right')

# Row 3: Sample images from each cluster
for c in range(K):
    ax = fig.add_subplot(gs[2, c])
    cluster_tiles = [t for t in tile_data if t['cluster'] == c]
    np.random.shuffle(cluster_tiles)
    
    # Create montage of 16 samples (4x4)
    montage = np.zeros((4*64, 4*64), dtype=np.uint8)
    for i in range(min(16, len(cluster_tiles))):
        img = cluster_tiles[i]['image']
        img_resized = cv2.resize(img, (64, 64))
        row, col = i // 4, i % 4
        montage[row*64:(row+1)*64, col*64:(col+1)*64] = img_resized
    
    ax.imshow(montage, cmap='gray')
    ax.set_title(f'Cluster {c}\n(n={len(cluster_tiles)})', color=CLUSTER_COLORS[c], fontweight='bold')
    ax.axis('off')
    
    # Add colored border
    for spine in ax.spines.values():
        spine.set_visible(True)
        spine.set_color(CLUSTER_COLORS[c])
        spine.set_linewidth(4)

plt.savefig(FIGURES_DIR / 'fig9_pipeline_summary.png', dpi=300, bbox_inches='tight')
plt.show()

---
## 7. Cluster Statistics Table

In [ ]:
print("=" * 80)
print("CLUSTER STATISTICS SUMMARY")
print("=" * 80)
print(f"{'Cluster':<10} {'Count':>8} {'%':>8} {'Intensity':>12} {'Area':>10} {'Solidity':>10} {'Circular':>10}")
print("-" * 80)

for c in range(K):
    cluster_tiles = [t for t in tile_data if t['cluster'] == c]
    n = len(cluster_tiles)
    pct = n / len(tile_data) * 100
    
    mean_int = np.mean([t['mean_intensity'] for t in cluster_tiles])
    mean_area = np.mean([t['area'] for t in cluster_tiles])
    mean_sol = np.mean([t['solidity'] for t in cluster_tiles])
    mean_circ = np.mean([t['circularity'] for t in cluster_tiles])
    
    print(f"C{c:<9} {n:>8} {pct:>7.1f}% {mean_int:>12.1f} {mean_area:>10.1f} {mean_sol:>10.3f} {mean_circ:>10.3f}")

print("=" * 80)
print(f"{'TOTAL':<10} {len(tile_data):>8}")
print(f"\nSilhouette Score: {silhouette_score(X_pca, labels):.3f}")

---
## 8. Export Results

In [ ]:
import csv

# Export to CSV
csv_path = OUTPUT_DIR / 'kmeans_clustering_results.csv'
fieldnames = ['array', 'filename', 'row', 'col', 'cluster', 'mean_intensity', 'area', 
              'solidity', 'circularity', 'max_gradient', 'filepath']

with open(csv_path, 'w', newline='') as f:
    writer = csv.DictWriter(f, fieldnames=fieldnames, extrasaction='ignore')
    writer.writeheader()
    for tile in tile_data:
        writer.writerow(tile)

print(f"Results exported to: {csv_path}")
print(f"Figures saved to: {FIGURES_DIR}")
print(f"\nGenerated figures:")
for f in sorted(FIGURES_DIR.glob('*.png')):
    print(f"  - {f.name}")

---
## 9. Throughput Analysis

In [ ]:
import time

# Benchmark throughput
print("Throughput Benchmarking...")
print("=" * 50)

# Time feature extraction
sample_tiles = tile_data[:100]
start = time.time()
for tile in sample_tiles:
    _ = extract_features(tile['image'])
feat_time = time.time() - start
feat_per_sec = 100 / feat_time

# Time clustering
start = time.time()
_ = KMeans(n_clusters=4, random_state=42, n_init=10).fit_predict(X_pca)
cluster_time = time.time() - start

total_atoms = len(tile_data)
estimated_total_time = (total_atoms / 100) * feat_time + cluster_time

print(f"Feature extraction: {feat_per_sec:.1f} atoms/second")
print(f"K-Means clustering: {cluster_time:.3f} seconds for {total_atoms} atoms")
print(f"\nTotal pipeline time: {estimated_total_time:.2f} seconds")
print(f"Throughput: {total_atoms / estimated_total_time:.1f} atoms/second")
print(f"\nScalability projection:")
print(f"  10,000 atoms: ~{10000 / feat_per_sec + cluster_time:.1f} seconds")
print(f"  100,000 atoms: ~{100000 / feat_per_sec + cluster_time * 10:.1f} seconds")